##### ARTI 560 - Computer Vision  
## Image Classification using Transfer Learning - Exercise 

### Objective

In this exercise, you will:

1. Select another pretrained model (e.g., VGG16, MobileNetV2, or EfficientNet) and fine-tune it for CIFAR-10 classification.  
You'll find the pretrained models in [Tensorflow Keras Applications Module](https://www.tensorflow.org/api_docs/python/tf/keras/applications).

2. Before training, inspect the architecture using model.summary() and observe:
- Network depth
- Number of parameters
- Trainable vs Frozen layers

3. Then compare its performance with ResNet and the custom CNN.

### Questions:

- Which model achieved the highest accuracy?
- Which model trained faster?
- How might the architecture explain the differences?

In [15]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input

In [16]:
# Load CIFAR-10
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

class_names = [
    "airplane","automobile","bird","cat","deer",
    "dog","frog","horse","ship","truck"
]

# Keep labels as integers (SparseCategoricalCrossentropy)
y_train = y_train.squeeze().astype("int64")
y_test  = y_test.squeeze().astype("int64")

# Convert images to float32
x_train = x_train.astype("float32")
x_test  = x_test.astype("float32")

In [17]:
# Data augmentation
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
], name="augmentation")

In [18]:
# Build EfficientNetB0 backbone (pretrained)
effnet_base = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)
effnet_base.trainable = False  # freeze first (feature extractor)

In [19]:
# Full model (preprocess inside model)
effnet_model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    data_augmentation,
    layers.Resizing(224, 224, interpolation="bilinear"),
    layers.Lambda(preprocess_input),          # IMPORTANT: correct for EfficientNetB0
    effnet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10)                          # logits
], name="cifar10_efficientnetb0")

effnet_model.summary()

Model: "cifar10_efficientnetb0"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augmentation (Sequential)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing_2 (Resizing)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_2 (Lambda)               │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,062,381 (15.50 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [20]:
# Compile + Train (frozen backbone)
effnet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=1),
]

history = effnet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 115s 148ms/step - accuracy: 0.6555 - loss: 1.0599 - val_accuracy: 0.8732 - val_loss: 0.3690 - learning_rate: 0.0010
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 105s 149ms/step - accuracy: 0.7930 - loss: 0.6063 - val_accuracy: 0.8868 - val_loss: 0.3356 - learning_rate: 0.0010
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 107s 153ms/step - accuracy: 0.8097 - loss: 0.5517 - val_accuracy: 0.8944 - val_loss: 0.3147 - learning_rate: 0.0010


In [21]:
# Test / Evaluate
test_loss, test_acc_r = effnet_model.evaluate(x_test, y_test, verbose=0)
print("EfficientNetB0 (frozen) test accuracy:", test_acc_r)
print("EfficientNetB0 (frozen) test loss    :", test_loss)

EfficientNetB0 (frozen) test accuracy: 0.8901000022888184
EfficientNetB0 (frozen) test loss    : 0.3281938135623932


In [22]:
# Fine-tune last layers
effnet_base.trainable = True
for layer in effnet_base.layers[:-30]:
    layer.trainable = False

print("Trainable layers in backbone:", sum(l.trainable for l in effnet_base.layers), "/", len(effnet_base.layers))

effnet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

history_ft = effnet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    verbose=1
)

test_loss_ft, test_acc_ft = effnet_model.evaluate(x_test, y_test, verbose=0)
print("EfficientNetB0 (fine-tuned) test accuracy:", test_acc_ft)
print("EfficientNetB0 (fine-tuned) test loss    :", test_loss_ft)

Trainable layers in backbone: 30 / 238


Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 143s 181ms/step - accuracy: 0.7671 - loss: 0.6982 - val_accuracy: 0.8824 - val_loss: 0.3535
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 126s 178ms/step - accuracy: 0.8113 - loss: 0.5533 - val_accuracy: 0.8958 - val_loss: 0.3163
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 126s 179ms/step - accuracy: 0.8260 - loss: 0.5046 - val_accuracy: 0.9012 - val_loss: 0.2944
EfficientNetB0 (fine-tuned) test accuracy: 0.8999999761581421
EfficientNetB0 (fine-tuned) test loss    : 0.29738980531692505


Resnet from lab2:

In [ ]:
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import preprocess_input


# Build ResNet50V2 backbone (pretrained)
resnet_base = ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)
resnet_base.trainable = False  # freeze first (feature extractor)

# Full model (preprocess inside model)
resnet_model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    data_augmentation,
    layers.Resizing(224, 224, interpolation="bilinear"),
    layers.Lambda(preprocess_input),          # IMPORTANT: correct for ResNet50V2
    resnet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10)                          # logits
], name="cifar10_resnet50v2")


# Compile + Train (frozen backbone)
resnet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=1),
]

history = resnet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

# Test / Evaluate
test_loss, test_acc_r = resnet_model.evaluate(x_test, y_test, verbose=0)
print("ResNet50V2 (frozen) test accuracy:", test_acc_r)
print("ResNet50V2 (frozen) test loss    :", test_loss)

# Fine-tune last layers
resnet_base.trainable = True
for layer in resnet_base.layers[:-30]:
    layer.trainable = False

print("Trainable layers in backbone:", sum(l.trainable for l in resnet_base.layers), "/", len(resnet_base.layers))

resnet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

history_ft = resnet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    verbose=1
)

test_loss_ft, test_acc_ft = resnet_model.evaluate(x_test, y_test, verbose=0)
print("ResNet50V2 (fine-tuned) test accuracy:", test_acc_ft)
print("ResNet50V2 (fine-tuned) test loss    :", test_loss_ft)

94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 171s 230ms/step - accuracy: 0.6762 - loss: 0.9283 - val_accuracy: 0.8608 - val_loss: 0.3872 - learning_rate: 0.0010
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 170s 242ms/step - accuracy: 0.8012 - loss: 0.5675 - val_accuracy: 0.8776 - val_loss: 0.3458 - learning_rate: 0.0010
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 173s 245ms/step - accuracy: 0.8181 - loss: 0.5236 - val_accuracy: 0.8808 - val_loss: 0.3441 - learning_rate: 0.0010
ResNet50V2 (frozen) test accuracy: 0.8783000111579895
ResNet50V2 (frozen) test loss    : 0.3578786253929138
Trainable layers in backbone: 30 / 190
Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 254s 343ms/step - accuracy: 0.7928 - loss: 0.6031 - val_accuracy: 0.8996 - val_loss: 0.2878
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 242s 344ms/step - accuracy: 0.8639 - loss: 0.3928 - val_accuracy: 0.9176 - val_loss: 0.2449
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 243s 345ms/step - accuracy: 0.8922 -

In [32]:
resnet_model.summary()


Model: "cifar10_resnet50v2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augmentation (Sequential)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing_3 (Resizing)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_3 (Lambda)               │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50v2 (Functional)         │ (None, 7, 7, 2048)     │    23,564,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │        20,490 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,513,312 (200.32 MB)

 Trainable params: 14,464,010 (55.18 MB)

 Non-trainable params: 9,121,280 (34.79 MB)

 Optimizer params: 28,928,022 (110.35 MB)

Custom CNN:

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_custom_cnn(input_shape=(32, 32, 3), num_classes=10):
    model = keras.Sequential([
        layers.Input(shape=input_shape),

        layers.Conv2D(32, 3, padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(32, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Dropout(0.25),

        layers.Conv2D(64, 3, padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.Conv2D(64, 3, padding="same", activation="relu"),
        layers.MaxPooling2D(),
        layers.Dropout(0.30),

        layers.Conv2D(128, 3, padding="same", activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),
        layers.Dropout(0.35),

        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(num_classes)  # logits
    ], name="cifar10_custom_cnn")

    return model

custom_cnn = build_custom_cnn()
custom_cnn.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


history_cnn = custom_cnn.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    verbose=1
)

test_loss_cnn, test_acc_cnn = custom_cnn.evaluate(x_test, y_test)
print("Custom CNN test acc:", test_acc_cnn)



Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 23s 19ms/step - accuracy: 0.2741 - loss: 2.1495 - val_accuracy: 0.4764 - val_loss: 1.5094
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.4656 - loss: 1.4791 - val_accuracy: 0.4948 - val_loss: 1.6217
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.5472 - loss: 1.2600 - val_accuracy: 0.6104 - val_loss: 1.1359
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6014 - loss: 1.1608
Custom CNN test acc: 0.6050000190734863


In [31]:
custom_cnn.summary()

Model: "cifar10_custom_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,001,408 (7.63 MB)

 Trainable params: 666,986 (2.54 MB)

 Non-trainable params: 448 (1.75 KB)

 Optimizer params: 1,333,974 (5.09 MB)

Compare models

In [30]:
# Collect and compare accuracies (update if you rename variables)
results = {
    "Custom CNN test acc": float(test_acc_cnn),
    "ResNet frozen test acc": float(test_acc_r) if 'test_acc_r' in globals() else None,
    "ResNet fine-tuned test acc": float(test_acc_ft) if 'test_acc_ft' in globals() else None,
    "EfficientNetB0 frozen test acc": float(test_acc_r) if 'test_acc_r' in globals() else None,
    "EfficientNetB0 fine-tuned test acc": float(test_acc_ft) if 'test_acc_ft' in globals() else None,
}
for k,v in results.items():
    print(f"{k}: {v}")


Custom CNN test acc: 0.6050000190734863
ResNet frozen test acc: 0.8783000111579895
ResNet fine-tuned test acc: 0.9192000031471252
EfficientNetB0 frozen test acc: 0.8783000111579895
EfficientNetB0 fine-tuned test acc: 0.9192000031471252


- Which model achieved the highest accuracy?

ResNet fine-tuned test acc: 0.9192000031471252
and 
EfficientNetB0 fine-tuned test acc: 0.9192000031471252
have same accuracy

- Which model trained faster?

Custom CNN


- How might the architecture explain the differences?

The custom CNN is relatively shallow.
ResNet50V2 and EfficientNetB0 is much deeper. Fine-tuning further improves adaptation to CIFAR-10.
But they are differ in total, trainable, and non-trainable params.
